Get spacy and gensim:

``conda install -c conda-forge spacy``\
``python -m spacy download en_core_web_lg``\
``conda install -c conda-forge gensim``


In [1]:
import pandas as pd

df = pd.read_csv('mbti_1.csv')
# split_posts = df['posts'].to_string().split('|||')

In [2]:
import numpy as np

# break up each row into its constituent sentences
# drop any row that has more than 50 sentences (these often have ||||||| in them)
l = df['posts'].values.tolist()
targets = df['type'].values.tolist()
new_list_1 = [x.split('|||') for x in l]

for i in range(len(new_list_1)):
    new_list_1[i].insert(0, targets[i])
    
new_list = [x for x in new_list_1 if len(x) <= 51]
df_mbti = pd.DataFrame(new_list)


In [3]:
from sklearn.model_selection import train_test_split

text_X_train, text_X_test, text_y_train, text_y_test = train_test_split(df_mbti.drop(columns=[0]),
                                                    df_mbti[0],
                                                    test_size = 0.2,
                                                    random_state = 123)

# reshape train data so each sentence is on its own row, remove Nones
text_X_train = text_X_train.to_numpy().reshape(-1,1)
text_y_train = np.c_[[text_y_train for i in range(50)]].T.reshape(-1,1)
text_train_data = np.c_[text_y_train, text_X_train]
text_train_data = text_train_data[text_train_data[:,1] != None]

text_X_test = text_X_test.to_numpy().reshape(-1,1)
text_y_test = np.c_[[text_y_test for i in range(50)]].T.reshape(-1,1)
text_test_data = np.c_[text_y_test, text_X_test]
text_test_data = text_test_data[text_test_data[:,1] != None]



In [20]:
y_test = [0 for x in text_test_data[:,0]]
y_1 = [np.zeros(4) for x in text_test_data[:,0]]
for i in range(len(y_test)):
    if text_test_data[i,0][0] == "I":
        y_test[i] += 1
        y_1[i][0] += 1
    if text_test_data[i,0][1] == "S":
        y_test[i] += 2
        y_1[i][1] += 1
    if text_test_data[i,0][2] == "T":
        y_test[i] += 4
        y_1[i][2] += 1
    if text_test_data[i,0][3] == "F":
        y_test[i] += 8
        y_1[i][3] += 1
        
y_train = [0 for x in text_train_data[:,0]]
y_2 = [np.zeros(4) for x in text_train_data[:,0]]
for i in range(len(y_train)):
    if text_train_data[i,0][0] == "I":
        y_train[i] += 1
        y_2[i][0] += 1
    if text_train_data[i,0][1] == "S":
        y_train[i] += 2
        y_2[i][1] += 1
    if text_train_data[i,0][2] == "T":
        y_train[i] += 4
        y_2[i][2] += 1
    if text_train_data[i,0][3] == "F":
        y_train[i] += 8
        y_2[i][3] += 1

y_2 = np.array(y_2)
y_1 = np.array(y_1)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,0])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,0]))

0.7526058514844368


In [30]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,1])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,1]))

0.8526465866340786


In [31]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,2])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,2]))

0.5962188196391345


In [32]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_2[:,3])

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_1[:,3]))

1.0


In [34]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text_train_data[:,1])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

X_1 = count_vect.transform(text_test_data[:,1])
X_2 = tfidf_transformer.fit_transform(X_1)
X_new_counts = count_vect.transform(text_test_data[:,1])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
y_pred = clf.predict(X_new_tfidf)
print(clf.score(X_new_tfidf, y_test))

0.3908297989600556
